**IMPORTING LIBRARIES**

In [2]:
import numpy as np 
import pandas as pd 
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib as mp
import bq_helper
import os
print(os.listdir("../input"))   


**IMPORTING A DATATSET ABOUT ASTRONAUTS, BECAUSE REALISTICALY I WANT TO BECOME AN ASTRONAUT NOT A COMPUTATIONAL DESIGNER...**

In [3]:
df = pd.read_csv("../input/astronauts.csv")
df

# df[df['Death Mission'].notnull()].shape

**ISOLATING SPACEWALKS COLUM**

In [4]:
swc = df["Space Walks"]
data = [go.Histogram(x=swc)]
data

#py.iplot(data, filename='basic histogram')

**PLOTTING SIMPLE HISTOGRAM ON NUMBER OF SPACEWALKS**

In [5]:
import matplotlib 
matplotlib.pyplot.hist(swc)
matplotlib.pyplot.hist(swc[swc > 0])

**REMOVING 0 COLUM ON HISTOGRAM**

In [6]:
matplotlib.pyplot.hist(swc[swc > 0])

**PLOTTING YEARS OF SPACEMISSIONS**

In [7]:
year = df['Year']
yearn = np.isnan(year) == False
#yearn
matplotlib.pyplot.hist(x = year[yearn])

**PLOTTING YEARS IN SEPERATE YEARS (INSTEAD OF BY EVERY 5 YEARS)**

In [8]:
year = df['Year']
yearn = np.isnan(year) == False
#yearn
matplotlib.pyplot.hist(x = year[yearn], bins = range(1955,2010))
year.sort_values()

**CHECKING ACCURACY OF HISTOGRAM**

In [9]:
allyears = year.sort_values()
year70s = allyears.iloc[range(60,100)]
year70s

**FIGURING OUT WHAT THE HELL A pandas.core.series.Series IS **

In [10]:
firstnames = df['Name'].str.split().apply(lambda x: x[0])
type(firstnames)

**WHAT IS THE MOST COMMON ASTRONAUT NAME AND HOW MANY DAYM MICHAELS ARE THERE?!?!?!**

In [11]:
df['firstnames'] = firstnames
FN = df.groupby('firstnames').count()
type(FN)
NFN = FN['Name']
NFNSORTED = NFN[NFN > 0].sort_values(ascending=False)
NFNSORTED
# # matplotlib.pyplot.hist(NFN[NFN > 1])

**TURNING A UGLY pandas.core.series.Series INTO A BEAUTIFUL DATAFRAME **

In [12]:
#ASTRONAMES = NFNSORTED.to_frame("")
ASTRONAMES = pd.DataFrame({'Astro names':NFNSORTED.index, 'Astro number':NFNSORTED.values})
ASTRONAMES

**A SIMPLE TRUTH CALCULATOR CHECKING THAT 16 IS ACTUALLY THE HIGHEST NUMBER**

In [13]:
count = 0
for value in NFN:
    if value > count:
        count = value 
print(count)

**IMPORTING AND CLEANING A DATASET CONTAINING THE MOST POPULAR NAMES IN THE USA** (because most of the astronauts are from the USA)

In [14]:

usa_names = bq_helper.BigQueryHelper(active_project="bigquery-public-data", dataset_name="usa_names")
usa_names

query = """SELECT name, sum(number) as number FROM `bigquery-public-data.usa_names.usa_1910_current` GROUP BY name order by number desc"""
names = usa_names.query_to_pandas_safe(query)
names.to_csv("usa_names.csv")
OVER1M = names[names['number'] > 10000]
OVER1M
# OVER1MN = OVER1M['name']
# OVER1MN
# OVER1MM = OVER1M['number']

**JOINING THE TWO DATASETS TOGETHER**

In [15]:
OVER1MN = OVER1M['name']
ASTRONAMES['world names'] = OVER1MN
OVER1MM = OVER1M['number']
ASTRONAMES['World number'] = OVER1MM
ASTRONAMES

**JOINING THE TWO DATASETS IN ACCORDANCE TO NAME AND DELETING STUPID NAMES LIKE C. or G. ** (because just joining them like above was super usless and a waste of time)

In [16]:
ASTRO_WITH_TOTAL = ASTRONAMES.set_index('Astro names').join(OVER1M.set_index('name'))
ASTRO_WITH_TOTAL
ASTRO_WITH_TOTAL_NAN = ASTRO_WITH_TOTAL[np.isfinite(ASTRO_WITH_TOTAL['number'])]
ASTRO_WITH_TOTAL_NAN

# ASTRO_WITH_TOTAL_NAN.index.values

# b = ASTRO_WITH_TOTAL_NAN['Astro names'] 
# b

# LS = []
# TN = ASTRO_WITH_TOTAL_NAN['number']
# TN
# for NUM in TN: 
#     ROUND = int(str(NUM)[:2])
#     LS.append(ROUND)
# LS

# for xx in LS

#     xo = (xo-xo) + 3 
#     LS.append(xo)
# LS
# VQ['Astro number']
# VQ = EQ[EQ['number'].map(len) < 2]

**FIRST ATTEMPT OF MAKING A PROBABILITY CALCULATOR THAT CALCULATES HOW LIKLEY U R TO END UP IN SPACE** (sooooo I tried to do this by deleting all but the first two digits in the number collum above to get a somewhat acurrate and also workable number. I then divided the Astro number and the number colums above together by index and multiplied them by 100 to get a percentage chance. There where a number of flaws in this method. 
1. not all numbers where in the millions so if i just got the first two digets of each number then it would defeat the whole shortcut esccence of the method because 2300000 is obviously larger than 230000 but the calculator would output 23 for both numbers. 
2. I was rounding to the nearest hundred thousand, super inacurate, to give a comparison, the ANZ stadium holds 80,000 people... 
3. the results came at a range of 20-30% and what does that even mean to a user that has no idea about the code... 

In [17]:
LS = []
TN = ASTRO_WITH_TOTAL_NAN['number']
TN
for NUM in TN: 
    ROUND = int(str(NUM)[:2])
    LS.append(ROUND)
LS

ASS = ASTRO_WITH_TOTAL_NAN['Astro number']
ASS

k = [(x / y) * 100 for x, y in zip(ASS, LS)]
k

**SECOND ATTEMPT, ALOT MORE  MATH... SHOUTOUT TO MY HIGHSCHOOL MATH TEACHER FOR TEACHING ME PROBABILITY** (bacicaly this gives you the most likley people or names that will become an astronaut. I add the lists Astro number and number above both seperatly then I found the % of each name from a sum of the whole list. I then subtract and divide the values in the two new lists to give me a positive or negative index. the higher the index the more likley you will end up in space... 

In [18]:
ALIST = ASTRO_WITH_TOTAL_NAN['Astro number']
SUM0 = sum(ALIST)

WLIST = ASTRO_WITH_TOTAL_NAN['number']
SUM1 = sum(WLIST)

PLIST0 = []
for py in ALIST:
    py = (py / SUM0) * 100
    PLIST0.append(py)
PLIST0

PLIST1 = []
for pi in WLIST:
    pi = (pi / SUM1) * 100
    PLIST1.append(pi)
PLIST1

CHANCECOLLUM = [(x - y) / y for x, y in zip(PLIST0, PLIST1)]
CHANCECOLLUM

NAMECOLLUM = ASTRO_WITH_TOTAL_NAN.index.values
# NAMECOLLUM = ASTRONAMES['Astro names']
# NAMECOLLUM

DAYTA = {'NAME': NAMECOLLUM, 'CHANCE': CHANCECOLLUM }
NAMEANDCHANCE = pd.DataFrame(data=DAYTA)
NAMEANDCHANCE

NAMEANDCHANCESORTED = NAMEANDCHANCE.sort_values(by=['CHANCE'], ascending=False)
NAMEANDCHANCESORTED

# VIBE = pd.DataFrame({'NAME':NAMECOLLUM.index, 'CHANCE':CHANCECOLLUM.values})
# VIBE


**CONVERTING A DATAFRAME TO A DICTIONARY**

In [19]:
dictionary = dict(zip(NAMECOLLUM, CHANCECOLLUM))
dictionary 


**VISULISING MY INDEX RANGE TO DECIDE MY CHANCE BRACKETS**

In [27]:
INDEX = NAMEANDCHANCESORTED['CHANCE']
INDEX

mp.pyplot.hist(INDEX, bins = range(-1,27))

# matplotlib.pyplot.hist(x = year[INDEX])
# year.sort_values()

# mp.plot(INDEX)
# mp.ylabel('some numbers')
# mp.show()


** THE OBVIOUS FLAW OF THIS ANAYLASIS**
Their have not been enough people who have gone to space, and their are too many names on earth. for example the number one position on who is most likley to go to space is Pierre, which is a very uncomon name in the USA, but their so happend to be two who went to space. If a person called Pierre was to put his name in the calculator it would say that he has a 25% chance of going to space. This means if we simply calculate how many Pierre's have been recorded on the USA names dataset ((2/25) x 100) we would get an answer of 8. soooooo their is not enough data.... or are all Peirrie's in the USA alians who are part of the illuminati and are working NASA and need to go to space or they will die??????  
